In [ ]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/boilers_drive/merged_df_30.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# path of CSV file
#file_path = 'merged_df_30.csv'

# the current script directory
# the current working directory
#current_dir = os.getcwd()

# the path to the CSV file in the parent directory
#parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
#file_path = os.path.join(parent_dir, 'merged_df_30.csv')

# reading CSV file
data = pd.read_csv(file_path)

In [ ]:
print("Imported data:")
print(data['blr_mod_lvl'].describe())

Imported data:
count    7.331988e+06
mean     2.258609e+00
std      8.733892e+00
min      0.000000e+00
25%      2.037800e-67
50%      5.652201e-17
75%      5.759105e-04
max      9.701499e+01
Name: blr_mod_lvl, dtype: float64


In [ ]:
# functions for plotting data
def plot_one(category, category_name):
    plt.plot(category, linestyle='-')
    plt.title("Plot of " + category_name)
    plt.ylabel(category_name)
    plt.show

def plot_dataset(dataset, column_names):
    for column in column_names:
        plt.plot(dataset[column], linestyle='-')
        plt.title("Plot of "+ column)
        plt.ylabel(column)
        #plt.grid(True)
        plt.show()

In [ ]:
# list of all column names
column_names = data.columns.tolist()

print("List of column names:")
print(column_names)

corr_columns = column_names.copy()
corr_columns.remove('time')
corr_columns.remove('house_id')
corr_columns.remove('nodata')

# list of column names for correlation
print("List of column names for correlation:")
print(corr_columns)

plot_columns = corr_columns.copy()
plot_columns.remove("month")
plot_columns.remove("day")
plot_columns.remove("hour")

# list of column names for plotting
print("List of column names for plotting:")
print(plot_columns)

List of column names:
['time', 'blr_mod_lvl', 'blr_t', 'heat', 'flame', 'water', 't_out', 't_ret', 't_r', 't_r_set', 'otc_cur', 't_set', 'otc_maxt', 'bypass', 'nodata', 'house_id', 'month', 'day', 'hour']
List of column names for correlation:
['blr_mod_lvl', 'blr_t', 'heat', 'flame', 'water', 't_out', 't_ret', 't_r', 't_r_set', 'otc_cur', 't_set', 'otc_maxt', 'bypass', 'month', 'day', 'hour']
List of column names for plotting:
['blr_mod_lvl', 'blr_t', 'heat', 'flame', 'water', 't_out', 't_ret', 't_r', 't_r_set', 'otc_cur', 't_set', 'otc_maxt', 'bypass']


In [ ]:
nodata_value = 0.0

filtered_nodata = data[data['nodata'] != nodata_value]

print(filtered_nodata)

Empty DataFrame
Columns: [time, blr_mod_lvl, blr_t, heat, flame, water, t_out, t_ret, t_r, t_r_set, otc_cur, t_set, otc_maxt, bypass, nodata, house_id, month, day, hour]
Index: []


In [ ]:
# Different houses (house ids) in data
houses = data['house_id'].unique()
print("Different houses in data:")
print(houses)
print("Number of different houses:")
print(len(houses))

Different houses in data:
['home2' 'home9' 'home13' 'home14' 'home34' 'home46' 'home55' 'home67'
 'home86' 'home93' 'home101' 'home106' 'home110' 'home43' 'home63'
 'home53' 'home79' 'home90' 'home95' 'home5' 'home17' 'home47' 'home51'
 'home65' 'home77' 'home89' 'home111' 'home114']
Number of different houses:
28


In [ ]:
# for plotting a category for all houses
def plot_houses_category(dataset, category):
    num_subplots = len(houses)

    subplot_width = 10
    subplot_height = 10
    total_width = num_subplots * subplot_width

    # Create the figure and axes
    fig, axes = plt.subplots(1, num_subplots, figsize=(total_width, subplot_height))

    # Iterate over categories and create subplots
    i=0
    for house in houses:
        # Get the data for the current category from each sub-dataset
        category_data = dataset[house][category]

        # Plot the data on the corresponding subplot
        ax = axes[i]
        ax.plot(category_data, linestyle='-', label=category)
        ax.set_title(house)
        ax.legend()
        #ax.grid(True)
        i=i+1

    # Adjust layout
    plt.tight_layout()
    plt.show()

Data Preprocessing

In [ ]:
data2 = data

# the categopries with which we will study the data

data2['absorption'] = data2['blr_t'] - data2['t_ret']
data2['insulation'] = data2['t_r'] - data2['t_out']

#print(data2)

In [ ]:
house_datasets2 = {}
for house in houses:
    house_datasets2[house] = data2[data2['house_id'] == house]

# Example
#print("House-dataset for house_id 'home34':")
#print(house_datasets2["home34"])

In [ ]:
def get_time_details(df):
    # Convert 'time' to datetime format
    df['time'] = pd.to_datetime(df['time'])
    # Sort the DataFrame by the time
    df = df.sort_values(by='time')
    first_time = df['time'].min()
    last_time = df['time'].max()
    # Calculate the difference between consecutive times
    time_diff = df['time'].diff()
    # Check if all differences are exactly 1 minute (60 seconds)
    all_one_minute_intervals = all(time_diff == pd.Timedelta(minutes=1))
    # Calculate the difference between consecutive times
    df['time_diff'] = df['time'].diff()
    # Identify intervals greater than 1 minute
    large_intervals = df[df['time_diff'] > pd.Timedelta(minutes=1)]
    # Extract times before and after the large intervals
    before_after_times = pd.DataFrame({
        'time_before': large_intervals['time'].shift(1),
        'time_after': large_intervals['time'],
        'time_interval': large_intervals['time_diff']
    }).dropna()
    time_details_temp = {
        'first_time': first_time,
        'last_time': last_time,
        'all_one_minute_intervals': all_one_minute_intervals,
        'before_after_times': before_after_times
    }
    return time_details_temp


# In time_details we will keep the details for all houses
time_details = {}
# Iterate for all houses
for house in houses:
  time_details[house] = get_time_details(house_datasets2[house])

#print()
#for house in houses:
#  print("House:", house)
#  print("First time:", time_details[house]['first_time'])
#  print("Last time:", time_details[house]['last_time'])
#  print("Are all intervals one minute:", time_details[house]['all_one_minute_intervals'])
#  print("Before and after times for large intervals (more than 1 minute):")
#  print(time_details[house]['before_after_times'])
#  print()

<ipython-input-12-2540233d0f9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-12-2540233d0f9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-12-2540233d0f9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [ ]:
for house in houses:
  duplicates = house_datasets2[house].duplicated(subset='time', keep=False)
  print("Duplicates for house", house, ":", duplicates.sum())

Duplicates for house home2 : 0
Duplicates for house home9 : 0
Duplicates for house home13 : 0
Duplicates for house home14 : 0
Duplicates for house home34 : 0
Duplicates for house home46 : 0
Duplicates for house home55 : 0
Duplicates for house home67 : 0
Duplicates for house home86 : 0
Duplicates for house home93 : 0
Duplicates for house home101 : 0
Duplicates for house home106 : 0
Duplicates for house home110 : 0
Duplicates for house home43 : 0
Duplicates for house home63 : 0
Duplicates for house home53 : 0
Duplicates for house home79 : 0
Duplicates for house home90 : 0
Duplicates for house home95 : 0
Duplicates for house home5 : 0
Duplicates for house home17 : 0
Duplicates for house home47 : 0
Duplicates for house home51 : 0
Duplicates for house home65 : 0
Duplicates for house home77 : 0
Duplicates for house home89 : 0
Duplicates for house home111 : 0
Duplicates for house home114 : 0


In [ ]:
# There is still error! The first and/or last time is not always the end and beginning of a day
def get_time_data(df):
  first_time_temp = df['time'].min()
  last_time_temp = df['time'].max()
  return first_time_temp, last_time_temp

first_times = []
last_times = []
for house in houses:
  first_time_temp, last_time_temp = get_time_data(house_datasets2[house])
  first_times.append(first_time_temp)
  last_times.append(last_time_temp)

time_data = pd.DataFrame({
    'house_id': houses,
    'first_time': first_times,
    'last_time': last_times
})
print(time_data)

   house_id          first_time           last_time
0     home2 2022-10-01 00:00:00 2023-04-30 23:59:00
1     home9 2022-10-01 00:00:00 2023-04-30 23:59:00
2    home13 2022-10-01 00:00:00 2023-04-30 23:59:00
3    home14 2022-11-04 19:19:00 2023-04-30 23:59:00
4    home34 2022-10-01 00:00:00 2023-04-30 23:59:00
5    home46 2022-10-01 00:00:00 2023-04-30 23:59:00
6    home55 2022-10-01 00:00:00 2023-04-30 23:59:00
7    home67 2022-10-01 00:00:00 2023-04-30 23:59:00
8    home86 2022-10-01 00:01:00 2023-04-30 23:59:00
9    home93 2022-10-01 00:00:00 2023-04-30 23:59:00
10  home101 2022-10-19 19:23:00 2023-04-30 23:59:00
11  home106 2022-11-13 11:39:00 2023-04-30 23:59:00
12  home110 2022-12-02 20:43:00 2023-04-30 23:59:00
13   home43 2022-10-01 00:00:00 2023-04-30 23:59:00
14   home63 2022-10-01 00:00:00 2023-04-30 23:59:00
15   home53 2022-10-01 00:00:00 2023-04-30 23:59:00
16   home79 2022-10-01 00:00:00 2023-04-30 23:59:00
17   home90 2022-10-01 00:00:00 2023-04-30 23:59:00
18   home95 

In [ ]:
# !! There are less time values than should be. There are no duplicates !!

# function which creates time entries with 1-minute intervals (fills in missing time values)
def fill_time(df, house_id):
    # Convert 'time' to datetime format
    df['time'] = pd.to_datetime(df['time'])
    # Sort the DataFrame by the time
    df = df.sort_values(by='time')

    # Determine the full time range: start at the beginning of the first day, end at the end of the last day
    start_date = df['time'].dt.floor('D').min()
    end_date = df['time'].dt.floor('D').max() + pd.Timedelta(days=1) - pd.Timedelta(minutes=1)

    # Create a complete range of time values with 1-minute intervals
    #full_time_range = pd.date_range(start=df['time'].min(), end=df['time'].max(), freq='T')
    # Create a complete range of time values with 1-minute intervals
    full_time_range = pd.date_range(start=start_date, end=end_date, freq='T')

    # Reindex the dataframe to include these full time values
    df = df.set_index('time').reindex(full_time_range).reset_index()
    # Rename the index column to 'time'
    df = df.rename(columns={'index': 'time'})
    df['house_id'] = house_id
    return df


for house in houses:
  house_datasets2[house] = fill_time(house_datasets2[house], house)

#print(house_datasets2)

<ipython-input-15-b439e9e90cef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-15-b439e9e90cef>:17: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_range = pd.date_range(start=start_date, end=end_date, freq='T')
<ipython-input-15-b439e9e90cef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-15-b439e9e90cef>:17: FutureWarning: 'T' is 

In [ ]:
# Now we find the new beginnings and endings
new_first_times = []
new_last_times = []
new_entries_num = []
for house in houses:
  new_first_time_temp, new_last_time_temp = get_time_data(house_datasets2[house])
  new_first_times.append(new_first_time_temp)
  new_last_times.append(new_last_time_temp)
  new_entries_num.append(len(house_datasets2[house]))

time_data['new_first_time'] = new_first_times
time_data['new_last_time'] = new_last_times
time_data['number_of_new_entries'] = new_entries_num
time_data['is_multiple_of_1440'] = time_data['number_of_new_entries'] % 1440 == 0
print(time_data.to_string(index=True))

   house_id          first_time           last_time new_first_time       new_last_time  number_of_new_entries  is_multiple_of_1440
0     home2 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59:00                 305280                 True
1     home9 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59:00                 305280                 True
2    home13 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59:00                 305280                 True
3    home14 2022-11-04 19:19:00 2023-04-30 23:59:00     2022-11-04 2023-04-30 23:59:00                 256320                 True
4    home34 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59:00                 305280                 True
5    home46 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59:00                 305280                 True
6    home55 2022-10-01 00:00:00 2023-04-30 23:59:00     2022-10-01 2023-04-30 23:59



---



---

**Find extreme values**

---



---



In [ ]:
# see if there are houses with unacceptable values of blr_mod_lvl
extreme_blr_mod_lvl = {}
for house in houses:
    data_temp = house_datasets2[house]
    extreme_blr_mod_lvl[house] = data_temp[data_temp['t_out'] > 100.0]
    extreme_blr_mod_lvl[house] = data_temp[data_temp['t_out'] < 0.0]

extreme_blr_mod_lvl_houses = []
not_extreme_blr_mod_lvl_houses = []
for house in houses:
    if not extreme_blr_mod_lvl[house].empty:
        extreme_blr_mod_lvl_houses.append(house)
    else:
        not_extreme_blr_mod_lvl_houses.append(house)

print("extreme_blr_mod_lvl found for houses with id :")
print(extreme_blr_mod_lvl_houses)
print("houses without extreme_blr_mod_lvl :")
print(not_extreme_blr_mod_lvl_houses)

extreme_blr_mod_lvl found for houses with id :
['home2', 'home14', 'home67', 'home101', 'home106', 'home110', 'home95', 'home47', 'home77', 'home89']
houses without extreme_blr_mod_lvl :
['home9', 'home13', 'home34', 'home46', 'home55', 'home86', 'home93', 'home43', 'home63', 'home53', 'home79', 'home90', 'home5', 'home17', 'home51', 'home65', 'home111', 'home114']


In [ ]:
# see if there are houses with extreme values of t_out
extreme_t_out = {}
for house in houses:
    data_temp = house_datasets2[house]
    extreme_t_out[house] = data_temp[data_temp['t_out'] > 35.0]
    extreme_t_out[house] = data_temp[data_temp['t_out'] < -5.0]

extreme_t_out_houses = []
not_extreme_t_out_houses = []
for house in houses:
    if not extreme_t_out[house].empty:
        extreme_t_out_houses.append(house)
    else:
        not_extreme_t_out_houses.append(house)

print("extreme_t_out found for houses with id :")
print(extreme_t_out_houses)
print("houses without extreme_t_out :")
print(not_extreme_t_out_houses)

extreme_t_out found for houses with id :
[]
houses without extreme_t_out :
['home2', 'home9', 'home13', 'home14', 'home34', 'home46', 'home55', 'home67', 'home86', 'home93', 'home101', 'home106', 'home110', 'home43', 'home63', 'home53', 'home79', 'home90', 'home95', 'home5', 'home17', 'home47', 'home51', 'home65', 'home77', 'home89', 'home111', 'home114']


In [ ]:
# see if there are houses with extreme values of absorption
extreme_absorption = {}

for house in houses:
    data_temp = house_datasets2[house]
    extreme_absorption[house] = data_temp[data_temp['absorption'] < 0.0]

extreme_absorption_houses = []
not_extreme_absorption_houses = []
for house in houses:
    if not extreme_absorption[house].empty:
        extreme_absorption_houses.append(house)
    else:
        not_extreme_absorption_houses.append(house)

print("extreme_absorption found for houses with id :")
print(extreme_absorption_houses)
print("houses without extreme_absorption :")
print(not_extreme_absorption_houses)

extreme_absorption found for houses with id :
['home2', 'home9', 'home13', 'home14', 'home34', 'home46', 'home55', 'home67', 'home86', 'home93', 'home101', 'home106', 'home110', 'home43', 'home63', 'home53', 'home79', 'home90', 'home95', 'home5', 'home17', 'home47', 'home65', 'home77', 'home89', 'home111', 'home114']
houses without extreme_absorption :
['home51']


In [ ]:
# Function which finds multiple consecutive values of absorption < 0
def get_consecutive_negatives(values):
  num = len(values)
  reversed_values = values[::-1]
  # initial consecutive_negatives are all 0
  reverse_consecutive_negatives = [0] * num
  for i in range(num):
    if reversed_values[i] < 0:
      # we only want the beginning of the negatives to have the length
      # all other negatives in the length will not be kept
      reverse_consecutive_negatives[i] = reverse_consecutive_negatives[i-1] + 1
      reverse_consecutive_negatives[i-1] = 0
  consecutive_negatives = reverse_consecutive_negatives[::-1]
  apoint_nan = 0
  for i in range(num):
    # if this is the beginning of multiple negatives then for the next values we will apoint NaN
    # we only check for more than 3 consecutive values (more than 3 minutes)
    if consecutive_negatives[i] > 3:
      apoint_nan =  consecutive_negatives[i]
    if apoint_nan > 0:
      values[i] = np.nan
      apoint_nan = apoint_nan - 1
  return values

In [ ]:
# for every value of 'absorption' we only accept positives
# if 'absorption' < 0 then it becomes NaN
for house in houses:
  # old attempt
  #house_datasets2[house]['absorption'] = house_datasets2[house]['absorption'].apply(lambda x: x if x > 0 else np.nan)
  # new attempt
  temp_data = house_datasets2[house]['absorption'].values
  temp_data = get_consecutive_negatives(temp_data)
  house_datasets2[house]['absorption'] = temp_data
# !! for a moment might be okay
# !! for many minutes (values) it is error
#print(house_datasets2)

In [ ]:
# for every value of 'blr_mod_lvl' we only accept within 0-100 range
for house in houses:
  # if 'blr_mod_lvl' < 0 then it becomes 0
  house_datasets2[house]['blr_mod_lvl'] = house_datasets2[house]['blr_mod_lvl'].apply(lambda x: x if x > 0 else 0.0)
  # if 'blr_mod_lvl' > 100 then it becomes 100
  house_datasets2[house]['blr_mod_lvl'] = house_datasets2[house]['blr_mod_lvl'].apply(lambda x: x if x < 100 else 100.0)
#print(house_datasets2)

In [ ]:
# the categories we will study
# first we will find anomalous values and normalize
anomalous_categories = ['blr_mod_lvl', 'absorption', 'insulation', 't_r_set', 't_out']

In [ ]:
def calculate_z_score(data, category):
    # Calculate mean and standard deviation of 'outside_temperature'
    mean_temp = data[category].mean()
    std_temp = data[category].std()

    z_score_name = 'z_score_' + category
    # Calculate Z-score for each data point
    data[z_score_name] = (data[category] - mean_temp) / std_temp
    return


def find_outliers_z_score(data, category):
    calculate_z_score(data, category)

    # threshold for Z-score (usually 3 standard deviations from the mean)
    z_threshold = 3
    if category=='blr_mod_lvl' :
        z_threshold = 8
    if category=='absorption' :
        z_threshold = 7
    if category=='insulation':
        z_threshold = 4
    if category=='t_out':
        z_threshold = 3
    if category=='t_out':
        z_threshold = 4

    z_score_name = 'z_score_' + category
    category_error = category + "_z_score_error"
    # Identify outliers based on Z-score threshold
    # Perform boolean indexing to flag anomalies based on the threshold
    data[category_error] = (data[z_score_name].abs() > z_threshold).astype(int)

    category_outliers = category + "_z_score_outliers"
    data[category_outliers] = data[category] * data[category_error]
    return

In [ ]:
for house in houses:
    for category in anomalous_categories:
        find_outliers_z_score(house_datasets2[house], category)

#print(house_datasets2)

In [ ]:
# plot two (or more) categories from a list for each house
def plot_houses_categories_list(dataset, categories):
    num_subplots = len(houses)

    subplot_width = 10
    subplot_height = 10
    total_width = num_subplots * subplot_width

    # Create the figure and axes
    fig, axes = plt.subplots(1, num_subplots, figsize=(total_width, subplot_height))

    # Iterate over categories and create subplots
    i=0
    for house in houses:
        # Get the data for the current category from each sub-dataset
        categories_data = {}
        for category in categories:
            categories_data[category] = dataset[house][category]

        # Plot the data on the corresponding subplot
        ax = axes[i]
        for category in categories:
            ax.plot(categories_data[category], linestyle='-', label=category)
        ax.set_title(house)
        ax.legend()
        #ax.grid(True)
        i=i+1

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot categories for houses

#for category in anomalous_categories:
#    plot_houses_category(house_datasets2, category)

In [ ]:
# Plot categories and z_score outliers for houses

#for category in anomalous_categories:
#    category_outliers = category + "_z_score_outliers"
#    categories = [category, category_outliers]
#    plot_houses_categories_list(house_datasets2, categories)



---



---

**Removed isolation forest**

---



---



In [ ]:
# Combine errors from multiple methods into one
def get_final_errors(data, category):
    z_score_error = category + "_z_score_error"
    #isolation_forest_error = 'isolation_forest_error_' + category
    combined_error = 'combined_error_' + category

    # Combine errors using logical operations
    #data[combined_error] = (data[z_score_error] & data[isolation_forest_error]).astype(int)
    #if category == 'blr_mod_lvl' :
    #    data[combined_error] = 0
    #if category == 'absorption' :
    #    data[combined_error] = (data[z_score_error] & data[isolation_forest_error]).astype(int)
    #if category == 'insulation' :
    #    data[combined_error] = (data[z_score_error] & data[isolation_forest_error]).astype(int)
    #if category == 't_r_set' :
    #    data[combined_error] = (data[z_score_error]).astype(int)
    #if category == 't_out' :
    #    data[combined_error] = (data[z_score_error] & data[isolation_forest_error]).astype(int)

    # !! for now we do not use isolation forest so there is only z-score error
    data[combined_error] = (data[z_score_error]).astype(int)

    combined_outliers = category + "_combined_outliers"
    data[combined_outliers] = data[category] * data[combined_error]

    # if combined_error == 1 then category = NaN
    # where keeps original value for combined_error == 0 (no error), else replaces with NaN
    data[category] = data[category].where(data[combined_error] == 0, np.nan)
    return

In [ ]:
for house in houses:
    for category in anomalous_categories:
        get_final_errors(house_datasets2[house], category)

#print(house_datasets2)

In [ ]:
# Plot categories and combined outliers for houses

#for category in anomalous_categories:
#    combined_outliers = category + "_combined_outliers"
#    # if we want categories (with their NaN values) and combined outliers
#    categories = [category, combined_outliers]
#    # if we do not care about combined outliers, only for categories with their NaN values
#    categories = [category]
#    plot_houses_categories_list(house_datasets2, categories)

In [ ]:
# Normalize with the average of the non-error previous and next values
def moving_average_imputation(data, category):
    normalized_category = 'normalized_' + category
    #combined_error = 'combined_error_' + category

    # Initialize a copy of the category data for modification
    df = data.copy()

    # Forward fill, fills NaN values with previous non-NaN
    df['ffill_category'] = df[category].ffill()

    # Backward fill, fills NaN values with next non-NaN
    df['bfill_category'] = df[category].bfill()

    # Calculate the mean of forward fill and backward fill where original values were NaN with priority on non-NaN values
    filled_category = df['ffill_category'].fillna(df['bfill_category']).add(df['bfill_category'].fillna(df['ffill_category'])) / 2

    data[normalized_category] = filled_category
    return

In [ ]:
for house in houses:
    for category in anomalous_categories:
        moving_average_imputation(house_datasets2[house], category)

#print(house_datasets2)



---



---

**Fill the NaN values for the other categories as well. We will keep them.**

---



---



In [ ]:
print(anomalous_categories)

normalized_categories = []
for category in anomalous_categories:
    normalized_categories.append('normalized_' + category)

print(normalized_categories)

['blr_mod_lvl', 'absorption', 'insulation', 't_r_set', 't_out']
['normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out']


In [ ]:
# plot normalized data

#for category in normalized_categories:
#    plot_houses_category(house_datasets2, category)

In [ ]:
# comparatively plot original data, error values and normalized data

#for category in anomalous_categories:
#    combined_outliers = category + "_combined_outliers"
#    normalized_category = 'normalized_' + category
#    categories = [category, combined_outliers, normalized_category]
#    plot_houses_categories_list(house_datasets2, categories)

In [ ]:
# the names of all the categories
# we choose one random house dataset, they all have the same categories
house = houses[0]
data_temp = house_datasets2[house]
column_names = data_temp.columns.tolist()

print("List of column names:")
print(column_names)

print("List of normalized category names:")
print(normalized_categories)

# the list of the final categories
final_categories = ['time']
final_categories.append('house_id')
for category in normalized_categories:
    final_categories.append(category)
final_categories.append('blr_t')
final_categories.append('heat')
final_categories.append('flame')
final_categories.append('water')
final_categories.append('t_ret')
final_categories.append('t_r')
final_categories.append('t_set')
final_categories.append('otc_cur')
final_categories.append('otc_maxt')
final_categories.append('bypass')

print("List of the final categories:")
print(final_categories)

List of column names:
['time', 'blr_mod_lvl', 'blr_t', 'heat', 'flame', 'water', 't_out', 't_ret', 't_r', 't_r_set', 'otc_cur', 't_set', 'otc_maxt', 'bypass', 'nodata', 'house_id', 'month', 'day', 'hour', 'absorption', 'insulation', 'z_score_blr_mod_lvl', 'blr_mod_lvl_z_score_error', 'blr_mod_lvl_z_score_outliers', 'z_score_absorption', 'absorption_z_score_error', 'absorption_z_score_outliers', 'z_score_insulation', 'insulation_z_score_error', 'insulation_z_score_outliers', 'z_score_t_r_set', 't_r_set_z_score_error', 't_r_set_z_score_outliers', 'z_score_t_out', 't_out_z_score_error', 't_out_z_score_outliers', 'combined_error_blr_mod_lvl', 'blr_mod_lvl_combined_outliers', 'combined_error_absorption', 'absorption_combined_outliers', 'combined_error_insulation', 'insulation_combined_outliers', 'combined_error_t_r_set', 't_r_set_combined_outliers', 'combined_error_t_out', 't_out_combined_outliers', 'normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_

In [ ]:
# Fill NaN values for categories
# these are the values of the final categories that were not normalized

# Fill with the average of the non-error previous and next values
def fill_nan_data(data, category, special=False):
    # Initialize a copy of the category data for modification
    df = data.copy()
    if (special==False): # normal categories, fill with mean of previous and next
        # Forward fill, fills NaN values with previous non-NaN
        df['ffill_category'] = df[category].ffill()
        # Backward fill, fills NaN values with next non-NaN
        df['bfill_category'] = df[category].bfill()
        # Calculate the mean of forward fill and backward fill where original values were NaN with priority on non-NaN values
        filled_category = df['ffill_category'].fillna(df['bfill_category']).add(df['bfill_category'].fillna(df['ffill_category'])) / 2
    else: # special categories, fill with previous only
        filled_category = df[category].ffill()
        # If NaN values at the beginning there is no "previous" value to fill from
        # There are still NaN values remaining
        # Fill with the default value
        default_value = 0
        filled_category = filled_category.fillna(default_value)
    data[category] = filled_category
    return

# special categories includes boolean and other categories that will be filled only with previous non-NaN value
special_categories = ['heat', 'flame', 'water', 'otc_cur', 'bypass']
# the remaining categories (after removing time and house_id)
remaining_categories = [x for x in final_categories if x not in special_categories]
remaining_categories.remove('time')
remaining_categories.remove('house_id')

print("Special categories:")
print(special_categories)
print("Remaining categories:")
print(remaining_categories)

for house in houses:
    for category in remaining_categories:
        fill_nan_data(house_datasets2[house], category, special=False)
    for category in special_categories:
        fill_nan_data(house_datasets2[house], category, special=True)

Special categories:
['heat', 'flame', 'water', 'otc_cur', 'bypass']
Remaining categories:
['normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out', 'blr_t', 't_ret', 't_r', 't_set', 'otc_maxt']


In [ ]:
# Concatenate all sub-datasets into one dataset
normalized_dataset = pd.concat(house_datasets2.values(), ignore_index=True)

# Select the desired columns from the combined dataset using the list of category names
final_dataset = normalized_dataset[final_categories]

# Print the final dataset
print(final_dataset)

                       time house_id  normalized_blr_mod_lvl  \
0       2022-10-01 00:00:00    home2            0.000000e+00   
1       2022-10-01 00:01:00    home2            0.000000e+00   
2       2022-10-01 00:02:00    home2            0.000000e+00   
3       2022-10-01 00:03:00    home2            0.000000e+00   
4       2022-10-01 00:04:00    home2            0.000000e+00   
...                     ...      ...                     ...   
7611835 2023-04-30 23:55:00  home114           2.871866e-119   
7611836 2023-04-30 23:56:00  home114           1.914578e-119   
7611837 2023-04-30 23:57:00  home114           1.276385e-119   
7611838 2023-04-30 23:58:00  home114           8.509234e-120   
7611839 2023-04-30 23:59:00  home114           5.672823e-120   

         normalized_absorption  normalized_insulation  normalized_t_r_set  \
0                     0.503910               7.457292                15.0   
1                     0.503910               7.455208                15.0   


In [ ]:
# !!! Are we finished ???
# looks like we are! No NaN values left by mistake.

# Identify NaN values
nan_mask = final_dataset.isna()
#print("NaN mask:\n", nan_mask)

# Get the count of NaN values in each column
nan_count = final_dataset.isna().sum()
print("\nNaN count in each column:\n", nan_count)

# List rows with NaN values
rows_with_nan = final_dataset[final_dataset.isna().any(axis=1)]
#print("\nRows with NaN values:\n", rows_with_nan)
print("\nHouses with NaN values:\n", rows_with_nan['house_id'].unique())

# List columns with NaN values
columns_with_nan = final_dataset.columns[final_dataset.isna().any()].tolist()
print("\nColumns with NaN values:\n", columns_with_nan)


NaN count in each column:
 time                      0
house_id                  0
normalized_blr_mod_lvl    0
normalized_absorption     0
normalized_insulation     0
normalized_t_r_set        0
normalized_t_out          0
blr_t                     0
heat                      0
flame                     0
water                     0
t_ret                     0
t_r                       0
t_set                     0
otc_cur                   0
otc_maxt                  0
bypass                    0
dtype: int64

Houses with NaN values:
 []

Columns with NaN values:
 []


In [ ]:
print(final_categories)
print(special_categories)
print(remaining_categories)

['time', 'house_id', 'normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out', 'blr_t', 'heat', 'flame', 'water', 't_ret', 't_r', 't_set', 'otc_cur', 'otc_maxt', 'bypass']
['heat', 'flame', 'water', 'otc_cur', 'bypass']
['normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out', 'blr_t', 't_ret', 't_r', 't_set', 'otc_maxt']


In [ ]:
# extra operations for improvement of data


# separating 'time' into components
# 'time' to datetime format
final_dataset['time'] = pd.to_datetime(final_dataset['time'])
# extracting components
final_dataset['year'] = final_dataset['time'].dt.year
final_dataset['month'] = final_dataset['time'].dt.month
final_dataset['day'] = final_dataset['time'].dt.day
final_dataset['day_of_year'] = final_dataset['time'].dt.dayofyear
final_dataset['hour'] = final_dataset['time'].dt.hour
final_dataset['minute'] = final_dataset['time'].dt.minute


# rounding values to reduce unecessary complexity for remaining_categories (they are boiler settings or temperature categories)
for category in remaining_categories:
    final_dataset[category] = final_dataset[category].round(2)


<ipython-input-42-5e386f7d6ae0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset['time'] = pd.to_datetime(final_dataset['time'])
<ipython-input-42-5e386f7d6ae0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset['year'] = final_dataset['time'].dt.year
<ipython-input-42-5e386f7d6ae0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [ ]:
# The path for the output CSV file
#output_csv_path = os.path.join(parent_dir, 'normalized_df.csv')
output_csv_path = '/content/drive/MyDrive/boilers_drive/normalized_df.csv'

# Export final dataset to a CSV file
final_dataset.to_csv(output_csv_path, index=False)